<a href="https://colab.research.google.com/github/Minseok-2001/FIFA22-DATA/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


Sun Sep 29 07:29:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

import os

DATA_DIR = '/content/drive/MyDrive/data'
import os
import ast
import pyarrow.parquet as pq
import pyarrow as pa

# DATA_DIR = 'C:\\Users\\zxcva\\Documents\\GitHub\\fragrance_ai\\data'
# def load_embeddings_from_parquet(file_path):
#     # Parquet 파일 읽기
#     return pd.read_parquet(file_path)
# # Parquet 파일에서 데이터 로드
# descriptions_df = load_embeddings_from_parquet(os.path.join(DATA_DIR, 'description_embeddings.parquet'))
# notes_df = load_embeddings_from_parquet(os.path.join(DATA_DIR, 'notes_embeddings.parquet'))
# reviews_df = load_embeddings_from_parquet(os.path.join(DATA_DIR, 'reviews_embeddings.parquet'))

# # perfumes_table.csv는 그대로 CSV로 읽기
# perfumes_df = pd.read_csv(os.path.join(DATA_DIR, 'perfumes_table.csv'))

# # 데이터 확인
# print(descriptions_df.head())
# print(notes_df.head())
# print(reviews_df.head())
# print(perfumes_df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_path = os.path.join(DATA_DIR, 'happy.jpg')


In [ ]:
# descriptions_df = pd.read_csv(os.path.join(DATA_DIR, 'description_embeddings.csv'))


In [ ]:
# reviews_df = pd.read_csv(os.path.join(DATA_DIR, 'reviews_embeddings.csv'))


In [ ]:
# notes_df = pd.read_csv(os.path.join(DATA_DIR, 'notes_embeddings.csv'))


In [ ]:
!pip install transformers mlflow torch torchvision pandas numpy scikit-learn pillow

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import ViTModel, ViTConfig, ViTImageProcessor
import mlflow
import mlflow.pytorch
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.0/563.0 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
class FragranceRecommendationModel(nn.Module):
    def __init__(self, embedding_dim, embedding_types, survey_dim):
        super(FragranceRecommendationModel, self).__init__()

        self.vit_config = ViTConfig.from_pretrained('google/vit-base-patch16-224')
        self.vit = ViTModel(self.vit_config)

        self.image_projection = nn.Linear(self.vit_config.hidden_size, embedding_dim)
        self.survey_projection = nn.Linear(survey_dim, embedding_dim)

        self.embedding_types = embedding_types
        self.type_embeddings = nn.ModuleDict({
            etype: nn.Linear(1536, embedding_dim) for etype in embedding_types
        })
        self.combiner = nn.Linear(len(embedding_types) * embedding_dim, embedding_dim)

        self.user_combiner = nn.Linear(2 * embedding_dim, embedding_dim)

    def forward(self, user_image, user_survey, perfume_embeddings):
        image_features = self.vit(user_image).last_hidden_state[:, 0]
        image_emb = self.image_projection(image_features)
        survey_emb = self.survey_projection(user_survey)
        user_emb = self.user_combiner(torch.cat([image_emb, survey_emb], dim=-1))

        perfume_embs = []
        for i, etype in enumerate(self.embedding_types):
            perfume_embs.append(self.type_embeddings[etype](perfume_embeddings[i]))

        combined_perfume_emb = self.combiner(torch.cat(perfume_embs, dim=-1))

        similarity = torch.sum(user_emb * combined_perfume_emb, dim=-1)
        return similarity


In [ ]:
class FragranceDataset(Dataset):
    def __init__(self, perfumes_df, descriptions_df, notes_df, reviews_df, transform=None):
        self.perfumes = perfumes_df
        self.descriptions = descriptions_df.set_index('url')
        self.notes = notes_df.set_index('url')
        self.reviews = reviews_df.set_index('url')
        self.transform = transform

    def __len__(self):
        return len(self.perfumes)

    def __getitem__(self, idx):
        perfume = self.perfumes.iloc[idx]
        url = perfume['url']

        image = torch.rand(3, 224, 224)
        if self.transform:
            image = self.transform(image)

        survey_data = torch.rand(10)

        embeddings = {}
        for key, df in [('description', self.descriptions), ('notes', self.notes), ('reviews', self.reviews)]:
            try:
                embeddings[key] = torch.tensor(df.loc[url, 'embedding_vector'], dtype=torch.float32)
            except KeyError:
                print(f"Warning: Missing {key} embedding for URL: {url}")
                embeddings[key] = torch.zeros(1536, dtype=torch.float32)  # 임베딩 차원에 맞게 조정

        preference = torch.tensor(np.random.choice([0, 1]), dtype=torch.float32)

        return image, survey_data, embeddings, preference

In [ ]:
def train(model, train_loader, val_loader, num_epochs, learning_rate, device):
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, surveys, embeddings, preferences = batch

            # 각 항목을 개별적으로 디바이스로 이동
            images = images.to(device)
            surveys = surveys.to(device)
            embeddings = {k: v.to(device) for k, v in embeddings.items()}
            preferences = preferences.to(device)

            optimizer.zero_grad()
            outputs = model(images, surveys, [embeddings['description'], embeddings['notes'], embeddings['reviews']])
            loss = criterion(outputs, preferences)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                images, surveys, embeddings, preferences = batch

                # 검증 데이터도 같은 방식으로 디바이스로 이동
                images = images.to(device)
                surveys = surveys.to(device)
                embeddings = {k: v.to(device) for k, v in embeddings.items()}
                preferences = preferences.to(device)

                outputs = model(images, surveys, [embeddings['description'], embeddings['notes'], embeddings['reviews']])
                loss = criterion(outputs, preferences)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {avg_val_loss:.4f}")

    return model

In [ ]:
def predict(model, image_path, survey_data, perfumes_df, descriptions_df, notes_df, reviews_df, device):
    model.eval()
    image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

    with torch.no_grad():
        image = Image.open(image_path)
        inputs = image_processor(images=image, return_tensors="pt").to(device)
        survey_tensor = torch.FloatTensor(survey_data).unsqueeze(0).to(device)

        similarities = []
        for _, perfume in perfumes_df.iterrows():
            url = perfume['url']
            perfume_embeddings = [
                torch.FloatTensor(descriptions_df.loc[url, 'embedding_vector']).unsqueeze(0).to(device),
                torch.FloatTensor(notes_df.loc[url, 'embedding_vector']).unsqueeze(0).to(device),
                torch.FloatTensor(reviews_df.loc[url, 'embedding_vector']).unsqueeze(0).to(device)
            ]
            similarity = model(inputs.pixel_values, survey_tensor, perfume_embeddings)
            similarities.append((url, similarity.item()))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:5]

In [ ]:
def get_notes_recommendation(recommended_perfumes, perfumes_df):
    all_notes = {'top': [], 'middle': [], 'base': []}
    for url, _ in recommended_perfumes:
        perfume = perfumes_df[perfumes_df['url'] == url].iloc[0]
        notes = eval(perfume['notes'])
        for note_type in all_notes:
            all_notes[note_type].extend(notes.get(note_type, []))

    recommended_notes = {
        note_type: list(set(notes))[:3] for note_type, notes in all_notes.items()
    }
    return recommended_notes

In [ ]:
def process_survey(survey_responses):
    processed_data = []

    # 1. 향의 강도
    processed_data.append(survey_responses['intensity'] / 5.0)

    # 2. 향의 지속성
    processed_data.append(survey_responses['longevity'] / 5.0)

    # 3. 사용 상황
    situations = ['daily', 'special', 'work']
    processed_data.extend([1 if s in survey_responses['occasions'] else 0 for s in situations])

    # 4. 선호 계절
    seasons = ['spring', 'summer', 'autumn', 'winter']
    processed_data.extend([1 if s in survey_responses['seasons'] else 0 for s in seasons])

    # 5. 선호 향 계열
    fragrance_families = ['floral', 'woody', 'citrus', 'oriental']
    processed_data.extend([1 if f in survey_responses['fragrance_families'] else 0 for f in fragrance_families])

    # 6. 성별
    genders = ['male', 'female', 'unisex']
    gender_index = genders.index(survey_responses['gender'])
    processed_data.extend([1 if i == gender_index else 0 for i in range(3)])

    # 7. 나이대
    age_groups = ['18-24', '25-34', '35-44', '45-54', '55+']
    age_index = age_groups.index(survey_responses['age_group'])
    processed_data.extend([1 if i == age_index else 0 for i in range(5)])

    # 8. 피부 타입
    skin_types = ['dry', 'oily', 'combination']
    skin_index = skin_types.index(survey_responses['skin_type'])
    processed_data.extend([1 if i == skin_index else 0 for i in range(3)])

    # 9. 선호 스타일
    styles = ['classic', 'modern', 'unique']
    style_index = styles.index(survey_responses['style_preference'])
    processed_data.extend([1 if i == style_index else 0 for i in range(3)])

    # 10. 이전 사용 브랜드
    brands = ['chanel', 'dior', 'gucci', 'ysl', 'tom_ford']  # 예시 브랜드 목록
    processed_data.extend([1 if b in survey_responses['previous_brands'] else 0 for b in brands])

    return processed_data

In [ ]:
def filter_perfumes(perfumes_df, descriptions_df, notes_df, reviews_df):
    valid_urls = set(perfumes_df['url']) & set(descriptions_df['url']) & set(notes_df['url']) & set(reviews_df['url'])
    return perfumes_df[perfumes_df['url'].isin(valid_urls)]

# 메인 함수에서 사용

In [ ]:
def load_data():
    perfumes_df = pd.read_csv(os.path.join(DATA_DIR, 'perfumes_table.csv'))
    descriptions_df = pd.read_parquet(os.path.join(DATA_DIR, 'description_embeddings.parquet'))
    notes_df = pd.read_parquet(os.path.join(DATA_DIR, 'notes_embeddings.parquet'))
    reviews_df = pd.read_parquet(os.path.join(DATA_DIR, 'reviews_embeddings.parquet'))
    return perfumes_df, descriptions_df, notes_df, reviews_df


In [ ]:

def prepare_dataset(perfumes_df, descriptions_df, notes_df, reviews_df):
    transform = transforms.Compose([
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    filtered_df = filter_perfumes(perfumes_df, descriptions_df, notes_df, reviews_df)
    dataset = FragranceDataset(filtered_df, descriptions_df, notes_df, reviews_df, transform=transform)
    return dataset

In [ ]:

def create_data_loaders(dataset):
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    print(f"Training on {len(train_dataset)} samples, validating on {len(val_dataset)} samples")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    return train_loader, val_loader

In [ ]:

def train_model(train_loader, val_loader):
    model = FragranceRecommendationModel(embedding_dim=256, embedding_types=['description', 'notes', 'reviews'], survey_dim=10)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trained_model = train(model, train_loader, val_loader, num_epochs=10, learning_rate=1e-4, device=device)
    return trained_model

In [ ]:

def make_recommendations(model, perfumes_df, descriptions_df, notes_df, reviews_df):
    test_survey_data = {
        'intensity': 4,
        'longevity': 3,
        'occasions': ['daily', 'work'],
        'seasons': ['spring', 'summer'],
        'fragrance_families': ['floral', 'fresh'],
        'gender': 'unisex',
        'age_group': '25-34',
        'skin_type': 'combination',
        'style_preference': 'modern',
        'previous_brands': ['chanel', 'jo_malone']
    }
    processed_survey = process_survey(test_survey_data)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    recommendations = predict(model, image_path, processed_survey, perfumes_df, descriptions_df, notes_df, reviews_df, device)

    print("Top 5 recommended perfumes:")
    for url, similarity in recommendations:
        perfume = perfumes_df[perfumes_df['url'] == url].iloc[0]
        print(f"- {perfume['name']} (Similarity: {similarity:.4f})")

    notes_recommendation = get_notes_recommendation(recommendations, perfumes_df)
    print("\nRecommended notes:")
    for note_type, notes in notes_recommendation.items():
        print(f"- {note_type.capitalize()} notes: {', '.join(notes)}")


In [21]:
def main():
    perfumes_df, descriptions_df, notes_df, reviews_df = load_data()
    dataset = prepare_dataset(perfumes_df, descriptions_df, notes_df, reviews_df)
    train_loader, val_loader = create_data_loaders(dataset)
    trained_model = train_model(train_loader, val_loader)
    make_recommendations(trained_model, perfumes_df, descriptions_df, notes_df, reviews_df)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training on 65982 samples, validating on 16496 samples
Training new model...


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Epoch 

OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 63.06 MiB is free. Process 456872 has 14.68 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 75.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)